In [48]:
#!/usr/bin/env python3
import random
import os

KEY = b"DUCTF{XXXXXXXXXXXXXXXXXXXXXXXXX}"
KEY_SIZE = 32
F = 2**14


class MyArrayGenerator:
    def __init__(self, key: bytes, n_registers: int = 128):
        self.key = key
        self.n_registers = n_registers

    def prepare(self):
        self.registers = [0 for _ in range(self.n_registers)]
        self.key_extension(self.key)

        self.carry = self.registers.pop()
        self.key_initialisation(F)

    def key_extension(self, key: bytes):
        if len(key) != KEY_SIZE:
            raise ValueError(f"Key length should be {KEY_SIZE} bytes.")

        for i in range(len(self.registers)):
            j = (4 * i) % KEY_SIZE
            subkey = key[j : j + 4]
            self.registers[i] = int.from_bytes(subkey)

    def key_initialisation(self, F: int):
        for _ in range(F):
            self.update()

    def shift(self):
        self.registers = self.registers[1:]

    def update(self):
        r0, r1, r2, r3 = self.registers[:4]

        self.carry ^= r1 if r2 > r3 else (r1 ^ 0xFFFFFFFF)

        self.shift()
        self.registers.append(self.registers[-1] ^ self.carry)

    def get_keystream(self) -> int:
        byte_index = random.randint(0, 3)
        byte_mask = 0xFF << (8 * byte_index)
        return (self.registers[-1] & byte_mask) >> (8 * byte_index)

    def encrypt(self, plaintext: bytes) -> bytes:
        self.prepare()

        ct = b""
        for b in plaintext:
            self.update()
            ct += int.to_bytes(self.get_keystream() ^ b)

        return ct

    def decrypt(self, ciphertext: bytes) -> bytes:
        return self.encrypt(ciphertext)


if __name__ == "__main__":
    random.seed(1234)
    cipher = MyArrayGenerator(KEY)

    plaintext = os.urandom(1280)
    print(f'plaintext = "{plaintext.hex()}"')

    ciphertext = cipher.encrypt(plaintext)
    print(f'ciphertext = "{ciphertext.hex()}"')


plaintext = "2725bebc0cf702411734877030020c35501885a8afc4805356e2ff1fcfdc3d91207d42ce9b4c97f1c473c9c52e21e8e67202763ac03fa47c94633913c6b8c1477702cd264ab0d1f70a636e976bb08769ebaf8bdad968b89d84f2a62e3676290a271751ea760f1563a180c802cfc71d448292840b0f0688e11c3cdb4f7dbf5f909ed25a455ba89b57962261ac4935f014087dde1ccb91920a7f59f63002c082454dfb86414cda1d6434cec46f5788375714949d87f6510ed15deef1ebf9e97ba33257b8145439d68991952b25249ba32d0d6eb37ac8dbf4f8f4ba962e6fd91bbb3d35b270a2642146386a3b717817be9c3000ddbb91b479773f76538e40feb8af472c918ba0909fa7864063cca614ae8ac84054634bfa620396dba5d4f28a47df5aa8d7175f98e263f68673e73026df34b524f5c62ccdefa1b622d3c3a60650077f78419b29fb2127a45bfcb81da782817a8167f1d6f7dcfafac1e0d28279fe8d0371015c9987ced224997fa9c05b24484b3d1c89ae62e31fffcbc76cb0ab578107c9e7e8b65b46fc81617b942c9b6fc2937d65bb4c9b48bae0b4a34a198eb71c6a1dc62d1ec11e35e2e3ff8f5266b7c75a4eb46aef8930f4f26881fe2ae4a04eef620117c69545e5b97443c562b496168014c1c60bd5f2bf04526ede9eab337c9937153766a4aa48dd2c3dc8075

TypeError: from_bytes() missing required argument 'byteorder' (pos 2)